In [6]:
import sys

!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests
import pandas as pd

You should consider upgrading via the '/Users/shriyabansal/.pyenv/versions/jupyter3/bin/python -m pip install --upgrade pip' command.
  Using cached requests-2.25.0-py2.py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.1-py2.py3-none-any.whl (136 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
You should consider upgrading via the '/Users/shriyabansal/.pyenv/versions/jupyter3/bin/python -m pip install --upgrade pip' command.


In [15]:
from dateutil.relativedelta import relativedelta
import requests
import json
import time
import dateutil
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [17]:
import datetime
end = datetime.date.today() 
start = end - relativedelta(years=1)
print('Start date: ' + str(start))
print('End date: ' + str(end))

Start date: 2019-11-12
End date: 2020-11-12


In [18]:
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]
months_in_range

[['2019', '12'],
 ['2020', '1'],
 ['2020', '2'],
 ['2020', '3'],
 ['2020', '4'],
 ['2020', '5'],
 ['2020', '6'],
 ['2020', '7'],
 ['2020', '8'],
 ['2020', '9'],
 ['2020', '10'],
 ['2020', '11']]

In [19]:
YOUR_API_KEY = 'qYusUdA1cSXCeKXXGVmiP7fd1aHg38EI'

In [20]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + YOUR_API_KEY
    try:
        response = requests.get(url, verify=False).json()
    except Exception:
        return None
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        print('Working on ' + str(date) + '...')
        csv_path = 'headlines/' + date[0] + '-' + date[1] + '.csv'
        if not os.path.exists(csv_path): # If we don't already have this month 
            response = send_request(date)
            if response is not None:
                df = parse_response(response)
                total += len(df)
                df.to_csv(csv_path, index=False)
                print('Saving ' + csv_path + '...')
    print('Number of articles collected: ' + str(total))

In [21]:
get_data(months_in_range)

Date range: ['2019', '12'] to ['2020', '11']
Working on ['2019', '12']...
Saving headlines/2019-12.csv...
Working on ['2020', '1']...
Saving headlines/2020-1.csv...
Working on ['2020', '2']...
Saving headlines/2020-2.csv...
Working on ['2020', '3']...
Saving headlines/2020-3.csv...
Working on ['2020', '4']...
Saving headlines/2020-4.csv...
Working on ['2020', '5']...
Saving headlines/2020-5.csv...
Working on ['2020', '6']...
Saving headlines/2020-6.csv...
Working on ['2020', '7']...
Saving headlines/2020-7.csv...
Working on ['2020', '8']...
Saving headlines/2020-8.csv...
Working on ['2020', '9']...
Saving headlines/2020-9.csv...
Working on ['2020', '10']...
Saving headlines/2020-10.csv...
Working on ['2020', '11']...
Saving headlines/2020-11.csv...
Number of articles collected: 78679


In [22]:
response

NameError: name 'response' is not defined